<a href="https://colab.research.google.com/github/December04/MachineLearningProgramming/blob/main/Week10/Assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GRU

In [14]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset/UCI HAR Dataset"

In [16]:
def load_file(filepath):
    return np.loadtxt(filepath)

def load_group(prefix, group="train"):
    signal_names = [
        "body_acc_x_", "body_acc_y_", "body_acc_z_",
        "body_gyro_x_", "body_gyro_y_", "body_gyro_z_",
        "total_acc_x_", "total_acc_y_", "total_acc_z_"
    ]
    data_dir = os.path.join(prefix, group, "Inertial Signals")
    arrays = []
    for s in signal_names:
        path = os.path.join(data_dir, f"{s}{group}.txt")
        arr = load_file(path)
        arrays.append(arr)
    stacked = np.stack(arrays, axis=-1)  # shape: (samples, 128, 9)
    return stacked

def load_labels(prefix, group="train"):
    path = os.path.join(prefix, group, f"y_{group}.txt")
    y = load_file(path).astype(int).flatten()
    return y - 1  # labels from 1~6 → 0~5

In [17]:
X_train = load_group(DATASET_PATH, "train")
X_test = load_group(DATASET_PATH, "test")
y_train = load_labels(DATASET_PATH, "train")
y_test = load_labels(DATASET_PATH, "test")

print("X_train:", X_train.shape)
print("X_test :", X_test.shape)

X_train: (7352, 128, 9)
X_test : (2947, 128, 9)


In [18]:
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]
scaler = StandardScaler()

X_train_2d = X_train.reshape(-1, n_features)
X_test_2d = X_test.reshape(-1, n_features)
scaler.fit(X_train_2d)

X_train_scaled = scaler.transform(X_train_2d).reshape(-1, n_timesteps, n_features)
X_test_scaled = scaler.transform(X_test_2d).reshape(-1, n_timesteps, n_features)

In [19]:
n_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, n_classes)
y_test_cat = to_categorical(y_test, n_classes)

In [20]:
def build_gru(input_shape, n_classes):
    model = Sequential()
    model.add(GRU(64, input_shape=input_shape, return_sequences=False))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_gru((n_timesteps, n_features), n_classes)
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        14,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,206 (75.02 KB)

 Trainable params: 19,078 (74.52 KB)

 Non-trainable params: 128 (512.00 B)

In [21]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint("best_har_gru.h5", monitor='val_loss', save_best_only=True)
]

In [22]:
history = model.fit(
    X_train_scaled, y_train_cat,
    validation_split=0.2,
    epochs=60,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/60


92/92 - 15s - 158ms/step - accuracy: 0.4047 - loss: 1.6608 - val_accuracy: 0.6125 - val_loss: 1.1933
Epoch 2/60


92/92 - 9s - 97ms/step - accuracy: 0.5778 - loss: 1.1006 - val_accuracy: 0.6472 - val_loss: 0.8652
Epoch 3/60


92/92 - 11s - 123ms/step - accuracy: 0.6598 - loss: 0.8264 - val_accuracy: 0.7933 - val_loss: 0.6429
Epoch 4/60


92/92 - 11s - 120ms/step - accuracy: 0.7419 - loss: 0.6039 - val_accuracy: 0.8144 - val_loss: 0.5400
Epoch 5/60


92/92 - 10s - 110ms/step - accuracy: 0.7631 - loss: 0.5810 - val_accuracy: 0.8158 - val_loss: 0.5076
Epoch 6/60


92/92 - 9s - 100ms/step - accuracy: 0.8085 - loss: 0.4312 - val_accuracy: 0.8736 - val_loss: 0.4230
Epoch 7/60


92/92 - 11s - 117ms/step - accuracy: 0.8509 - loss: 0.3570 - val_accuracy: 0.9164 - val_loss: 0.3202
Epoch 8/60


92/92 - 11s - 118ms/step - accuracy: 0.9143 - loss: 0.2422 - val_accuracy: 0.9109 - val_loss: 0.2859
Epoch 9/60


92/92 - 9s - 96ms/step - accuracy: 0.9354 - loss: 0.1972 - val_accuracy: 0.9320 - val_loss: 0.2129
Epoch 10/60
92/92 - 12s - 125ms/step - accuracy: 0.9435 - loss: 0.1588 - val_accuracy: 0.9225 - val_loss: 0.2475
Epoch 11/60
92/92 - 11s - 116ms/step - accuracy: 0.9447 - loss: 0.1533 - val_accuracy: 0.9082 - val_loss: 0.2324
Epoch 12/60
92/92 - 19s - 203ms/step - accuracy: 0.9509 - loss: 0.1345 - val_accuracy: 0.8967 - val_loss: 0.2492
Epoch 13/60
92/92 - 11s - 119ms/step - accuracy: 0.9497 - loss: 0.1338 - val_accuracy: 0.9191 - val_loss: 0.2609
Epoch 14/60
92/92 - 11s - 115ms/step - accuracy: 0.9486 - loss: 0.1324 - val_accuracy: 0.9069 - val_loss: 0.2577
Epoch 15/60
92/92 - 9s - 97ms/step - accuracy: 0.9444 - loss: 0.1461 - val_accuracy: 0.9096 - val_loss: 0.2391
Epoch 16/60
92/92 - 11s - 119ms/step - accuracy: 0.9493 - loss: 0.1322 - val_accuracy: 0.9245 - val_loss: 0.2365
Epoch 17/60
92/92 - 11s - 123ms/step - accuracy: 0.9549 - loss: 0.1172 - val_accuracy: 0.9171 - val_loss: 0.2398

In [23]:
loss, acc = model.evaluate(X_test_scaled, y_test_cat, verbose=0)
print(f"✅ Test loss: {loss:.4f}, accuracy: {acc:.4f}")

✅ Test loss: 0.4369, accuracy: 0.8856


In [24]:
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step

📊 Classification Report:
              precision    recall  f1-score   support

           0     0.9168    0.9113    0.9141       496
           1     0.8569    0.8896    0.8729       471
           2     0.8943    0.9667    0.9291       420
           3     0.8098    0.7719    0.7904       491
           4     0.8330    0.8346    0.8338       532
           5     1.0000    0.9497    0.9742       537

    accuracy                         0.8856      2947
   macro avg     0.8851    0.8873    0.8857      2947
weighted avg     0.8862    0.8856    0.8855      2947

Confusion Matrix:
[[452  13  30   1   0   0]
 [ 32 419  18   1   1   0]
 [  9   5 406   0   0   0]
 [  0  24   0 379  88   0]
 [  0   1   0  87 444   0]
 [  0  27   0   0   0 510]]
